<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/Toyama_Uni_2_4_LLM_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.必要なライブラリのインストール

まず、Hugging FaceのTransformersライブラリをインストールします。

In [ ]:
!pip install transformers

#2.モデルとトークナイザーのロード

次に、rinna/japanese-gpt2-smallモデルと対応するトークナイザーをロードします。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# トークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-small", use_fast=False)
tokenizer.do_lower_case = True  # 一部のバグ回避のための設定

# モデルのロード
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/454M [00:00<?, ?B/s]

#3.モデルのデバイスへの配置

Google ColabでGPUを使用する場合、モデルをGPUに配置します。

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32000, bias=False)
)

#4.テキスト生成の実行

以下のコードで、モデルを使用して日本語のテキスト生成を行います。

In [ ]:
# 入力テキストの設定
input_text = "良い天気ですね"

# トークナイズとテンソル変換
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# テキスト生成
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# 生成されたテキストのデコード
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


良い天気ですね昨日は、いつもの通り、午後から、実家に、今年も、お世話になっている、 で、昨日から、京都と青森へ、友達とのお茶会をしたり、晩酌を頂きました 昨日の、会社の先輩の、私の友人のおかげで、とても、良い買い物ができました まず、皆が、飲み会で行きたくて行った、私の母校、立命館大学に 学生生活では、色んなイベントがあります
